In [14]:
%pip install git+https://github.com/MarcusLoppe/meshgpt-pytorch.git
%pip install matplotlib

  Cloning https://github.com/MarcusLoppe/meshgpt-pytorch.git to /tmp/pip-req-build-fvr3dke2
  Running command git clone --filter=blob:none --quiet https://github.com/MarcusLoppe/meshgpt-pytorch.git /tmp/pip-req-build-fvr3dke2
  Resolved https://github.com/MarcusLoppe/meshgpt-pytorch.git to commit b5e74674973de4a97431561d7723d9b29629a695
  Preparing metadata (setup.py) ... done

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [15]:
from pathlib import Path 
import gc    
import torch
import os
import torch  
from meshgpt_pytorch import (
    MeshTransformerTrainer,
    MeshAutoencoderTrainer,
    MeshAutoencoder,
    MeshTransformer,MeshDataset
)
from meshgpt_pytorch.data import ( 
    derive_face_edges_from_faces
)   


In [16]:
%cd /root/text-to-mesh

/root/text-to-mesh


In [17]:
autoencoder = MeshAutoencoder( 
        decoder_dims_through_depth =  (128,) * 6 + (192,) * 12 + (256,) * 24 + (384,) * 6,    
        dim_codebook = 192,  
        dim_area_embed = 16,
        dim_coor_embed = 16, 
        dim_normal_embed = 16,
        dim_angle_embed = 8,
    
    attn_decoder_depth  = 4,
    attn_encoder_depth = 2
    ).to("cuda")     

# pkg = torch.load("./MeshGPT-autoencoder_0.38_x1s.pt") 
# autoencoder.load_state_dict(pkg['model'])
# for param in autoencoder.parameters():
#     param.requires_grad = True

In [18]:
dataset = MeshDataset.load("./objverse_250f_45.9M_3086_labels_53730_10_min_x1_aug.npz")  
dataset2 = MeshDataset.load("./objverse_250f_229.7M_3086_labels_268650_10_min_x5_aug.npz")
dataset.data.extend(dataset2.data)  
dataset2 = MeshDataset.load("./shapenet_250f_2.2M_84_labels_2156_10_min_x1_aug.npz")  
dataset.data.extend(dataset2.data)  
dataset2 = MeshDataset.load("./shapenet_250f_21.9M_84_labels_21560_10_min_x10_aug.npz")  
dataset.data.extend(dataset2.data) 
dataset.sort_dataset_keys()
print("length", len(dataset.data))

def format_value(value):
    if value >= 1_000_000_000:
        return f"{value / 1_000_000_000:.1f}B"
    elif value >= 1_000_000:
        return f"{value / 1_000_000:.1f}M"
    else:
        return f"{value}"

tokens = 0
for item in dataset.data:
    tokens += len(item['faces']) * 6 
total_tokens = format_value(tokens)
print("Tokens:", total_tokens)

[MeshDataset] Loaded 53730 entrys
[MeshDataset] Created from 53730 entrys


KeyboardInterrupt: 

In [ ]:
# pkg = torch.load("checkpoints/mesh-autoencoder.ckpt.epoch_5_avg_loss_0.17483_recon_0.3439_commit_-0.6763.pt") 
# autoencoder.load_state_dict(pkg['model'])
# for param in autoencoder.parameters():
#     param.requires_grad = True

In [ ]:
batch_size=16 # The batch size should be max 64.
grad_accum_every =4
# So set the maximal batch size (max 64) that your VRAM can handle and then use grad_accum_every to create a effective batch size of 64, e.g  16 * 4 = 64
learning_rate = 1e-3 # Start with 1e-3 then at stagnation around 0.35, you can lower it to 1e-4.

autoencoder.commit_loss_weight = 0.2 # Set dependant on the dataset size, on smaller datasets, 0.1 is fine, otherwise try from 0.25 to 0.4.
autoencoder_trainer = MeshAutoencoderTrainer(model =autoencoder ,warmup_steps = 10, dataset = dataset, num_train_steps=100,
                                             batch_size=batch_size,
                                             grad_accum_every = grad_accum_every,
                                             learning_rate = learning_rate,
                                             checkpoint_every_epoch=5)
 
loss = autoencoder_trainer.train(480, diplay_graph= True)   

Epoch 1/480:   0%|          | 1/21631 [00:00<45:39,  7.90it/s, commit_loss=-0.654, loss=0.21, recon_loss=0.341]

Epoch 1/480: 100%|██████████| 21631/21631 [40:31<00:00,  8.90it/s, commit_loss=-0.465, loss=0.273, recon_loss=0.366] 


Epoch 1 average loss: 0.2940775129664573 recon loss: 0.3854: commit_loss -0.4568


Epoch 2/480: 100%|██████████| 21631/21631 [40:33<00:00,  8.89it/s, commit_loss=-0.649, loss=0.239, recon_loss=0.369]  


Epoch 2 average loss: 0.2704318430853212 recon loss: 0.3785: commit_loss -0.5404


...

Epoch 4/480: 100%|██████████| 21631/21631 [40:51<00:00,  8.82it/s, commit_loss=-0.617, loss=0.235, recon_loss=0.358] 


Epoch 4 average loss: 0.24313940016614718 recon loss: 0.3686: commit_loss -0.6271          avg loss speed: 0.029210834821537385 epochs left: 1.95


Epoch 5/480: 100%|██████████| 21631/21631 [40:32<00:00,  8.89it/s, commit_loss=0.626, loss=0.886, recon_loss=0.761]   


Epoch 5 average loss: 0.23568133945427833 recon loss: 0.3665: commit_loss -0.6539          avg loss speed: 0.019689524599969532 epochs left: 3.27


Epoch 6/480:  49%|████▉     | 10642/21631 [20:27<21:43,  8.43it/s, commit_loss=-0.962, loss=0.151, recon_loss=0.343]

In [ ]:
import datetime
import torch
now_utc = datetime.datetime.now(datetime.timezone.utc).isoformat()
now_safe = now_utc.replace(":", "_")
pkg = dict( model = autoencoder.state_dict(), )
filename = f"./MeshGPT-autoencoder_{now_safe}_0.38_x1s.pt"
torch.save(pkg, filename)
import torch
import random
from tqdm import tqdm 
from meshgpt_pytorch import mesh_render 

min_mse, max_mse = float('inf'), float('-inf')
min_coords, min_orgs, max_coords, max_orgs = None, None, None, None
random_samples, random_samples_pred, all_random_samples = [], [], []
total_mse, sample_size = 0.0, 200

random.shuffle(dataset.data)
autoencoder = autoencoder.to("cuda")
for item in tqdm(dataset.data[:sample_size]):
    item['faces'] = item['faces'].to("cuda")
    item['vertices'] = item['vertices'].to("cuda")
    item['face_edges'] = item['face_edges'].to("cuda")
    codes = autoencoder.tokenize(vertices=item['vertices'], faces=item['faces'], face_edges=item['face_edges']) 
    
    codes = codes.flatten().unsqueeze(0)
    codes = codes[:, :codes.shape[-1] // autoencoder.num_quantizers * autoencoder.num_quantizers] 
 
    coords, mask = autoencoder.decode_from_codes_to_faces(codes)
    orgs = item['vertices'][item['faces']].unsqueeze(0)

    mse = torch.mean((orgs.view(-1, 3).cpu() - coords.view(-1, 3).cpu())**2)
    total_mse += mse 

    if mse < min_mse: min_mse, min_coords, min_orgs = mse, coords, orgs
    if mse > max_mse: max_mse, max_coords, max_orgs = mse, coords, orgs
 
    if len(random_samples) <= 30:
        random_samples.append((coords, mask)) 
    else:
        all_random_samples.extend([ random_samples])
        random_samples, random_samples_pred = [], []

print(f'MSE AVG: {total_mse / sample_size:.10f}, Min: {min_mse:.10f}, Max: {max_mse:.10f}')    
mesh_render.save_rendering(f'./mse_rows.obj', all_random_samples)

 

100%|██████████| 200/200 [00:12<00:00, 16.50it/s]


MSE AVG: 0.0011568634, Min: 0.0000151837, Max: 0.0201407764
[Save_rendering] Saved at ./mse_rows.obj


In [ ]:
transformer.conditioner.text_models[0].model.to("cuda")
transformer = transformer.to("cuda")

dataset.embed_texts(transformer,1)

In [ ]:
 
torch.cuda.empty_cache()
gc.collect()  
 
batch_size = 2
grad_accum_every =4     
trainer = MeshTransformerTrainer(model = transformer,warmup_steps = 10,grad_accum_every=grad_accum_every,num_train_steps=100, dataset = dataset, 
                                  accelerator_kwargs = {"mixed_precision" : "fp16"}, optimizer_kwargs = { "eps": 1e-7} , 
                                 learning_rate = 1e-3, batch_size=batch_size ,checkpoint_every_epoch = 25) 
loss = trainer.train(35, stop_at_loss = 0.00005)   
 

In [ ]:
# pkg = dict( model = autoencoder.state_dict(), ) 
# torch.save(pkg, str(f"./MeshGPT-autoencoder_0.38_x1s.pt"))

In [ ]:
import torch
import random
from tqdm import tqdm 
from meshgpt_pytorch import mesh_render 

min_mse, max_mse = float('inf'), float('-inf')
min_coords, min_orgs, max_coords, max_orgs = None, None, None, None
random_samples, random_samples_pred, all_random_samples = [], [], []
total_mse, sample_size = 0.0, 200

random.shuffle(dataset.data)
autoencoder = autoencoder.to("cuda")
for item in tqdm(dataset.data[:sample_size]):
    item['faces'] = item['faces'].to("cuda")
    item['vertices'] = item['vertices'].to("cuda")
    item['face_edges'] = item['face_edges'].to("cuda")
    codes = autoencoder.tokenize(vertices=item['vertices'], faces=item['faces'], face_edges=item['face_edges']) 

    codes = codes.flatten().unsqueeze(0)
    codes = codes[:, :codes.shape[-1] // autoencoder.num_quantizers * autoencoder.num_quantizers] 
 
    coords, mask = autoencoder.decode_from_codes_to_faces(codes)
    orgs = item['vertices'][item['faces']].unsqueeze(0)

    mse = torch.mean((orgs.view(-1, 3).cpu() - coords.view(-1, 3).cpu())**2)
    total_mse += mse 

    if mse < min_mse: min_mse, min_coords, min_orgs = mse, coords, orgs
    if mse > max_mse: max_mse, max_coords, max_orgs = mse, coords, orgs
 
    if len(random_samples) <= 30:
        random_samples.append((coords, mask)) 
    else:
        all_random_samples.extend([ random_samples])
        random_samples, random_samples_pred = [], []

print(f'MSE AVG: {total_mse / sample_size:.10f}, Min: {min_mse:.10f}, Max: {max_mse:.10f}')
mesh_render.save_rendering(f'.\mse_rows.obj', all_random_samples)

  0%|          | 0/200 [00:00<?, ?it/s]

100%|██████████| 200/200 [00:12<00:00, 15.97it/s]


MSE AVG: 0.0004646061, Min: 0.0000161871, Max: 0.0137998303
[Save_rendering] Saved at .\mse_rows.obj


In [ ]:
now_utc = datetime.datetime.now(datetime.timezone.utc).isoformat()
now_safe = now_utc.replace(":", "_")
filename = f"./MeshGPT-autoencoder_{now_safe}.pt"
pkg = dict( model = autoencoder.state_dict(), ) 
torch.save(pkg, filename)